In [54]:
from datasets import load_dataset
import pandas as pd
import numpy as np
from itertools import islice
from urllib.parse import urlparse
import ast


In [2]:
# Load filtered an streamed FineWeb 2 dataset
dataset = load_dataset("HuggingFaceFW/fineweb-2", split="train", data_dir="data/dan_Latn", streaming=True)

In [3]:
def get_batches(ds, batch_size):
    iterator = iter(ds)
    while True:
        batch = list(islice(iterator, batch_size))
        if not batch:
            break
        yield batch


def domain_distribution(df):
    if "domain" not in df.columns:
        raise ValueError("DataFrame does not contain a 'domain' column.")
    
    distribution = df["domain"].value_counts().sort_values(ascending=False)
    return distribution

def save_text_label(text, label, path="fineweb2_data.csv"):
    try:
        df = pd.read_csv(path)
        df["label"] = df["label"].apply(ast.literal_eval)  # convert label column from str to list
    except FileNotFoundError:
        df = pd.DataFrame(columns=["text", "label"])

    match = df[df["text"] == text]

    if not match.empty:
        idx = match.index[0]
        labels = df.at[idx, "label"]
        if label not in labels:
            labels.append(label)
            df.at[idx, "label"] = labels
    else:
        new_row = pd.DataFrame([{"text": text, "label": [label]}])
        df = pd.concat([df, new_row], ignore_index=True)

    df.to_csv(path, index=False)

In [25]:
Batch_Size = 1000
Batch_Iterations = 13
Remove_Domains = ["www.texaspoker.dk"]

batch_generator = get_batches(dataset, batch_size=Batch_Size)

# Initialize storage
all_records = pd.DataFrame()
all_domains = []

# Process in batches
for i in range(Batch_Iterations):
    print(f"Batch {i+1} of {Batch_Iterations}")
    batch = next(batch_generator)

    records = [{
        "text": example["text"],
        "minhash_cluster_size": example.get("minhash_cluster_size"),
        "url": example.get("url")
    } for example in batch]

    temp_Records = pd.DataFrame(records)
    temp_Records["domain"] = temp_Records["url"].apply(lambda url: urlparse(url).netloc if url else None)

    # Filter out domains
    filtered_Records = temp_Records[~temp_Records["domain"].isin(Remove_Domains)].reset_index(drop=True)

    all_records = pd.concat([all_records, filtered_Records], ignore_index=True)
    all_domains.extend(filtered_Records["domain"].unique())



    

Batch 1 of 13
Batch 2 of 13
Batch 3 of 13
Batch 4 of 13
Batch 5 of 13
Batch 6 of 13
Batch 7 of 13
Batch 8 of 13
Batch 9 of 13
Batch 10 of 13
Batch 11 of 13
Batch 12 of 13
Batch 13 of 13


In [26]:
print(domain_distribution(all_records))
print("Total Domains:",len(all_domains))

domain
da.wikipedia.org                                                       929
www.andersen.sdu.dk                                                    366
ekstrabladet.dk                                                        311
www.bt.dk                                                              292
www.agoda.com                                                          278
www.tripadvisor.dk                                                     200
www.jetcost.dk                                                         174
www.dagens.dk                                                          165
deborah.dk                                                             158
www.denstoredanske.dk                                                  149
europa.eu                                                              115
www.wimdu.dk                                                           110
www.avisen.dk                                                          110
www.trustpilot.co.

In [7]:
def browse_samples_by_domain(all_domains, all_records):
    pd.set_option('display.max_colwidth', 100)  # limit text preview
    pd.set_option('display.max_rows', False)    # show all rows if needed

    for domain in all_domains:
        domain_samples = all_records[all_records["domain"] == domain]

        print(f"\n--- Domain: {domain} ---")
        if domain_samples.empty:
            print("No samples available.")
        else:
            sample = domain_samples.sample(n=1).iloc[0]
            display(sample)
            print(f"Sample text:\n{sample['text']}\n")

        user_input = input("Press ENTER to continue or 'q' to quit: ").strip().lower()
        if user_input == 'q':
            print("Exiting...")
            break

In [ ]:
browse_samples_by_domain(all_domains, all_records)

In [ ]:
# 6, good, 31, good, 183, good, 1490 good, 2509, good, 164, good kinda, 198, good, 

In [52]:
check_domain = "www.ikea.com"

# Show all rows
pd.set_option('display.max_rows', None)

# Limit text column width (e.g. to 100 characters)
pd.set_option('display.max_colwidth', 500)

# Filter and display
all_records[all_records["domain"] == check_domain]

,text,minhash_cluster_size,url,domain
119,"Hvis du foretrækker det, kan vi samle dine møbler for dig.\nMed monteringsservice får du hjælp af dygtige montører, som er eksperter i at montere alle IKEA produkter.\nDu kan få hjælp til alt fra opmåling, møbelmontage af reoler, skabe, kommoder, kontormøbler og til ophængning af lamper, gardiner og billeder.\nDet spiller ingen rolle om du skal have samlet få eller mange møbler, bor i en ét værelses eller har en stor virksomhed.\nHos IKEA har du en tidsubegrænset fortrydelses- og returret.\n...",2034,http://www.ikea.com/dk/da/catalog/products/S79873725/,www.ikea.com
753,"Hos IKEA har du en tidsubegrænset fortrydelses- og returret.\nSkifter du mening kan du på hvilket som helst tidspunkt vælge at fortryde dit køb hos IKEA.\nBetingelserne er blot, at varen tilbageleveres til IKEA i original emballage, samt at du medbringer og kan fremvise en gyldig kvittering.\nDu skal selv betale for fragtomkostningerne i forbindelse med tilbageleveringen til et IKEA varehus.\nProduct Recycling Fund\nThe price of this item includes a contribution to a Product recycling fund t...",83293,http://www.ikea.com/dk/da/catalog/products/90237951/,www.ikea.com
849,"Hos IKEA har du en tidsubegrænset fortrydelses- og returret.\nSkifter du mening kan du på hvilket som helst tidspunkt vælge at fortryde dit køb hos IKEA.\nBetingelserne er blot, at varen tilbageleveres til IKEA i original emballage, samt at du medbringer og kan fremvise en gyldig kvittering.\nDu skal selv betale for fragtomkostningerne i forbindelse med tilbageleveringen til et IKEA varehus.\nForskellige vægmaterialer kræver forskellige typer skruer/rawlplugs. Brug de skruer/rawlplugs, der p...",2,http://www.ikea.com/dk/da/catalog/products/S79885969/,www.ikea.com
955,"ALLE AFDELINGER:\n-\nDækkeservietter, der pynter på bordet\nDækkeservietter er en fast del af borddækningen – de er dekorative og beskytter dit bord mod ridser og spildpletter. Nogle IKEA dækkeservietter, f.eks. PANNÅ, fås i forskellige farver, så du kan finde nogle, der passer eller står i kontrast til dugen eller dit service. Skal det være helt enkelt, så vælg ORDENTLIG, der er nem at tørre af. Alle dækkeservietter beskytter dit bord og dæmper støj fra tallerkener og bestik, så du kan konc...",40,http://www.ikea.com/dk/da/catalog/categories/departments/Textiles/20539/,www.ikea.com
1313,"Nu er det blevet nemmere for firmaer og institutioner at handle i IKEA.\nMed en IKEA erhvervskonto har I som erhvervsdrivende mulighed for at købe i samtlige IKEA varehuse i Danmark, med op til 30 dages rentefri kredit. For hvert køb modtager I helt automatisk en faktura, ønsker I at få oplyst saldoen, så kontakt IKEA Erhverv.\nI kan downloade et ansøgningsskema direkte fra vores hjemmeside eller hente et ansøgningsskema i varehuset.",3,http://www.ikea.com/ms/da_DK/customer_service/ikea_card/IKEA_business_card.html,www.ikea.com
1359,"Hos IKEA har du en tidsubegrænset fortrydelses- og returret.\nSkifter du mening kan du på hvilket som helst tidspunkt vælge at fortryde dit køb hos IKEA.\nBetingelserne er blot, at varen tilbageleveres til IKEA i original emballage, samt at du medbringer og kan fremvise en gyldig kvittering.\nDu skal selv betale for fragtomkostningerne i forbindelse med tilbageleveringen til et IKEA varehus.\nKan kombineres med MICKE høj tilbygningsdel. Hvis opbevaringssystemet suppleres med en tilbygningsde...",16,http://www.ikea.com/dk/da/catalog/products/10221614/,www.ikea.com
1644,Kolde retter/smørrebrød\nProduct Recycling Fund\nThe price of this item includes a contribution to a Product recycling fund to ensure that waste electrical and electronic equipment is collected and recycled in a responsible manner.\nIKEA Aarhus\nRestaurant & Bistro\nMandag til søndag kl. 9.30 - 19.30\nMorgenmad serveres i tidsrummet kl. 9.30 - 11\nAlle dage kl. 10.00 - 20.,14,http://www.ikea.com/dk/da/store/aarhus/restaurant/ICID=DKAAR_MODUL1_RESTAURANT01092013,www.ikea.com
1736,"Nyheder, lænestole, sofaborde,

In [49]:
all_records.iloc[7205]["text"] # 2888, 2988, 1211

'Kort over den globale luftforurening\nDet fremgår tydeligt af dette billede, hvilke lokaliteter i verden, som er mest plaget af luftforurening. Billedet er baseret på 18 måneders observationer fra den europæiske miljøsatellit Envisat.\nKortet viser fordelingen af forureningen med kvælstofoxider i atmosfæren. Man kan se, at der er høje koncentrationer af denne type forurening knyttet til større byer tværs over hele Nordamerika og Europa såvel som andre steder, som f.eks. Mexico City i Mellemamerika og sydafrikanske kulfyrede kraftværker i Sydafrika, der er placeret i nærheden af hinanden i den østlige del af landets Highveld-højslette.\n"Desuden findes der en meget høj koncentration over det nordøstlige Kina. Også tværs over det sydøstlige Asien og store dele af Afrika kan der ses kvælstofdioxid produceret af afbrænding af biomasse. Spor fra skibe er synlige visse steder: Se på Det Røde Hav og Det Indiske Ocean mellem Indiens sydspids og Indonesien. Skibes skorstene sender disse steder

In [46]:
labels = ["None", "Minimal", "Basic", "Good", "Excellent"]

save_text_label(all_records.iloc[7185]["text"], labels[4])

In [55]:
pd.read_csv("fineweb2_data.csv")  # Load the saved CSV to check the saved labels

,text,label
0,"Du kan bruge visningstilstande, når du vil læse dokumentet eller flytte rundt i det.\nHvis du vil læse dokumentet uden at anstrenge øjnene mere end højst nødvendigt og bruge værktøjer, der er specielt beregnet til læsning af dokumenter, skal du vælge læselayout (læselayout: En visning, der er beregnet til læsning af dokumenter på en computerskærm. Dokumentet får en størrelse, der passer til skærmen, de fleste værktøjslinjer er fjernet, men der er stadig kommandoer til navigering, kommenterin...",['Good']
1,"\n\nDen indre barriere…\nPerspektiv Nr. 2-2000\nSkær ned på fedtet – spis flere kulhydrater – men hvilke?\nTatjana Hejgaard Vasilaras, M. Sc., Ph.D. stud. & Arne Astrup, professor dr.med., Forskningsinstitut for Human Ernæring.\nAnbefalingerne er klare: Vi skal skære ned på fedtet og erstatte det med fødevarer, der har et højere indhold af kulhydrat og protein.\nVæk med piskefløden, osten, de fede kød- og pålægsprodukter, chipsene og fedtstofferne. Ind med brød, pasta, ris, grøntsager og de ...",['Good']
2,"\n\nDe hører ligesom Cirrus og Cirrostratus til de høje skyer, hvilket vil sige, at de typisk befinder sig i tre til otte kilometers højde i polare områder, fem til 13 kilometers højde i tempererede områder og seks til 18 kilometer i tropiske områder.\nCirrocumulus kan dannes i klar luft og kan være resultat af omdannelsen af Cirrus- eller Cirrostratusskyerne. De kan også dannes som resultat af formindskelse af de elementer som en Altocumulusflage eller et lag består af.",['Good']
3,"Pisk æggehvider helt stive og vend forsigtigt de hakkede mandler, flormelis og essens heri.\nSæt dejen i små skefulde på en plade med bagepapir, der skal være god afstand mellem kagerne.\nPynt med en halv mandel på hver og bag dem ved 180 grader i ca. 12 minutter - til de er svagt lysebrune. Køl dem af på en rist.",['Good']
4,"Rens din skærm billigst og bedst\nHvordan renser man mest effektivt sin computer- eller tv-skærm? Fotograf og forfatter til blandt andet Politikens store bog om digital fotografering Thomas Nykrog har undersøgt emnet.\nBomuld og lunkent vand\nHan har blandt andet talt med mange computerfirmaer, og det viser sig, at en skærm renses bedst med en bomuldsklud, der ikke fnuller og almindeligt lunkent vand. De firmaer som selv fabrikerer rensevæske til skærme, anbefaler selvfølgelig, at man bruger...",['Good']
5,"Pil løgene og hak dem fint. Skyl bladselleristænglerne og skær dem ud i mindre stykker. Steg løgene klare i smør i en gryde med bladselleri, tilsæt skyllede timianblade eller tørret timian, paprika og majskerner.\nHæld mælk i og kog suppen ca. 15 min. Tilsæt salt. Blend suppen, til den er helt jævn og glat (kom den evt. igennem en grov si).\nSkær kyllingekødet i små tern. Kom kyllingekødet i den kogende suppe, og skru ned, så suppen småkoger i ca. 10 min., til kyllingekødet er gennemkogt.\nT...",['Good']
6,"Tomaterne skoldes, flås og skæres i små tern (fjern også de fleste kerner). De hakkede tomater puttes i en gryde og bringes i kog sammen med bouillon, basilikum eller timian, laurbærbladet, salt, peber og chili. Suppen bringes i kog og småkoger ca. 15 min. Herefter fjernes basilikum/timian og laurbærbladet og suppen pureres med en stavblender eller i en blender/food-processor. Put Creme Fine og kødboller i den purerede suppe, varm den op og smag evt. til med krydderier.\nServeres med et par ...",['Good']
7,"Varm ovnen op til 200o . Udrør gæren med vand. Tilsæt olie, salt og 2/3 af melet. Når melet slipper skålen vendes dejen ud på bordet og æltes igennem med mere mel.\nSæt dejen til hævning dækket med et fugtigt klæde 20 min.\nÆlt dejen igennem. Del den i to. Rul dem meget tyndt ud på et meldrysset bord.\nLæg bundene på plade beklædt med bagepapir og prik dem med en gaffel.\nPensl kanterne med lidt olie. Forbag dem uden fyld 10-15 min.\nRiv alle grøntsager fint evt. med foodprocessor.\nSvits gr...",['Good']
8,"Anden renses for indmad og skylles. Gnides herefter godt med salt og peber indvendig og u